In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import glob
file_names = glob.glob('/content/drive/MyDrive/earning_call_experiments/*.txt')
print(len(file_names))

56


In [ ]:
files_text = []
for txt_file in file_names:
    with open(txt_file, 'r') as f:
        files_text.append(f.read())

In [ ]:
!pip install stanza

     |████████████████████████████████| 432 kB 2.8 MB/s 
     |████████████████████████████████| 170 kB 38.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=f593a7734211b22bbbb625337c893a27cef6d2369fb317e27ae0bef77cda694b
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
Successfully built emoji


In [ ]:
import stanza
from tqdm.notebook import tqdm

stanza.download('en')

2021-10-17 12:46:30 INFO: Downloading default packages for language: en (English)...


2021-10-17 12:46:44 INFO: Finished downloading models and saved to /root/stanza_resources.


In [ ]:
len(file_names)

56

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')
filtered_sentences = []
sentence_filenames = []
for file_text, filename in tqdm(zip(files_text, file_names), total=len(file_names)):
    doc = nlp(file_text)
    for sentence in doc.sentences:
        temp_sentence = sentence.text
        if '?' not in temp_sentence:
            for ent in sentence.ents:
                if 'PERSON' in ent.type:
                    temp_sentence = temp_sentence.replace(ent.text, '')
            filtered_sentences.append(temp_sentence)
            sentence_filenames.append(filename)

2021-10-12 14:03:24 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2021-10-12 14:03:24 INFO: Use device: gpu
2021-10-12 14:03:24 INFO: Loading: tokenize
2021-10-12 14:03:35 INFO: Loading: ner
2021-10-12 14:03:35 INFO: Done loading processors!


  0%|          | 0/56 [00:00<?, ?it/s]

In [ ]:
print(len(filtered_sentences))
print(len(sentence_filenames))

30408
30408


In [ ]:
import pandas as pd
df = pd.DataFrame({
    'filename': [s.split('/')[-1] for s in sentence_filenames],
    'sentence': filtered_sentences
})
df.head()

,filename,sentence
0,MSFT_q1_2018.txt,"Operator\nGreetings, and welcome to the First ..."
1,MSFT_q1_2018.txt,"As a reminder, this conference is being recorded."
2,MSFT_q1_2018.txt,"I would like to turn the call over to , Genera..."
3,MSFT_q1_2018.txt,", please proceed."
4,MSFT_q1_2018.txt,Chris Suh - Microsoft Corp.


In [ ]:
keyword_terms = {
    'Digital Product': 'smart product, connected product, software as a service, Saas,\
        platform as a service, Paas, platform, product as a service',

    'Digital Customer Experience': 'digital experience, customer experience, CX, user experience, UX,\
        user journey, customer journey, digital engagement, customer\
        engagement, personalization, personalisation, digital marketing,\
        recommendation, market place, marketplace, e-commerce platform,\
        digital service, digital services, e-service, online chat, chatbot, app,\
        digital chnnel, omnichannel',

    'Digital Operations': 'process automation, process mining, process analytics, process\
        optimization, efficiency, cost saving, cost reduction, reduce cost,\
        reducing cost, automation, predictive maintenance, ERP, supply chain,\
        logistics, operations',

    'Digital Business Model': 'business model, new market, new segment, monitization, Saas,\
        software as a service, on-demand, product as a service, value\
        proposition, freemium, subscription, marketplace, ad-revenue, ads,\
        peer-to-peer, two-sided, double-sided',

    'Enablers': 'digital strategy, digital business strategy, digital transformation\
        strategy, governance, priositization, prioritization, digital vision,\
        digital leadership, leadership support, leadership buy-in,\
        communication, digital goals, data scientist, data analyst machine\
        learning engineer, developer, coder, programmer, chief digital officer,\
        CDO, head of digital transformation, head of digital, product manager,\
        product owner, cross-functional, scrum master, agile coach, innovation\
        manager, Data lake, data warehouse, middle ware, enterprise\
        architecture, digital tools, digital workplace, digital integration, chat,\
        video call, CRM, ERP, service oriented architecture, SOA',

    'Practices': 'Agile, scrum, MVP, minimum viable product, sprint, design thinking,\
        business experiment, DevOps, epic, feature, user story, product\
        owner, product manager, collaboration, cross functional, crossfunctional, A/B testing, exploratory data analysis, data analysis,\
        decision support system, dashboard, hypothesis testing, experimental\
        design, product metrics, user metrics, usage metrics, click through rate,\
        conversion rate, click stream, digital marketing, customer\
        segmentation, risk modelling, simulation, decision analytics, decision\
        support system, project management, digital skills, digital leadership,\
        transformation, data analysis, social media management, social\
        listening, user research, UX research, UX design, UI design,\
        programming, coding, lean startup, experimentation, incubator,\
        accelerator, innovation lab, digital lab, digital transformation, open\
        innovation, design thinking, design sprint, digitalization, digitalisation,\
        digitization, digital technology',

    'AI': 'AI, artificial intelligence, NLP, natural language processing,\
        natural language understanding, NLU, natural language generation,\
        NLG, speech recognition, sentiment analysis, speech to text, text to\
        speech, deep learning, machine learning, ML, neural network,\
        algorithm, generative adversarial network, GANs, supervised learning,\
        unsupervised learning, reinforcement learning, semi-supervised\
        learning, active learning, self learning, transfer learning, back\
        propogation, tensorflow, Salesforce Einstein, IBM Watson, kaggle, AI\
        as a service, Microsoft azure ML, AutoML, autonomous vehicles,\
        computer vision, image recognition, pattern recognition, cognitive\
        computing, predictive analytics, predictive maintenance, algorithmic\
        trading, clustering, dimensionality reduction, t-sne, PCA, principal\
        component analysis, chatbot, bot, RPA, robotic process\
        automation, matrix factorization, collaborative filtering, recommender\
        system, recommendation engine, graph mining, graph theory, cortana,\
        alexa, google assistant',

    'Cloud computing': 'cloud computing, cloud native, cloudless, distributed cloud, distributed\
        computing, clustered computing, hybrid cloud, platform as a service,\
        edge computing, cloud api, google cloud, azure cloud, aws cloud,\
        software as a service, cloud applications, cloud, GPU, HPC\
        management, cloud storage, elasticity, elastic computing, the cloud,\
        data platform',

    'IoT': 'Internet of things, IoT, industrial internet, IIoT, embedded device,\
        embedded sensor, digital twin, digital thread, building information\
        modelling, BIM, connected devices, connected sensors, IoE, internet\
        of everything, smart machines, connected machines, wearable, cyber\
        physical systems, machine to machine, connected factory, model\
        based definition',

    'Virtual reality': 'VR, virtual reality, immersive technologies, mixed reality',

    'Augmented reality': 'AR, augmented reality, immersive technologies, mixed reality',

    'Robotics': 'robots, humanoid, drone, drones, smart robots, smart warehouse, smart\
        spaces, Lidar, computer vision, UAV, autonomous vehicles, swarm\
        robots, industrial robot, robotics, automation',

    'Analytics': 'analytics, business intelligence, optimization, exploratory data\
        analysis, data science, augmented analytics, descriptive analytics,\
        descriptive statistics, prescriptive analytics, predictive, inference,\
        inferential, customer segmentation, correlation, data visualization, data\
        storytelling, text analytics, data lake, data warehouse, big data, social\
        analytics, network analytics, network mining, network analysis',

    'Mobile': 'mobile, smart phone, mobile app, mobile application, mobile platform,\
        mobile solution, mobile technology',

    'Social': 'social media, social network, content marketing',
    
    '3D printing': '3D print, additive manufacturing, 3D scan, material jetting,\
        stereolithography, bioprint, bioprinted organ, Fused deposition\
        modeling, Digital Light Processing, Selective Laser Sintering,\
        Selective laser melting, Laminated object manufacturing, Digital Beam\
        Melting',

    'Blockchain': 'blockchain, distributed ledger, decentralized, smart contracts,\
        cryptocurrency, ICO, initial coin offering, asset tokenization'
}

In [ ]:
def search_keywords(sentence):
    categories = set()
    keywords = set()
    for key, value in keyword_terms.items():
        for keyword in value.split(','):
            if " " + keyword.strip().lower() + " " in sentence.lower():
                keywords.add(keyword.strip())
                categories.add(key)
    return pd.Series({
        'keywords': list(keywords),
        'categories': list(categories)
    })

In [ ]:
tqdm.pandas()
df[['keywords', 'categories']] = df.sentence.progress_apply(lambda s: search_keywords(s))

  0%|          | 0/30408 [00:00<?, ?it/s]

In [ ]:
df['contains_keyword'] = df.keywords.progress_apply(lambda l: True if len(l) > 0 else False)

  0%|          | 0/30408 [00:00<?, ?it/s]

In [ ]:
df[df.contains_keyword==True]

,filename,sentence,keywords,categories,contains_keyword
11,MSFT_q1_2018.txt,As we discuss our Q1 results and share our out...,"[cloud, communication]","[Cloud computing, Enablers]",True
28,MSFT_q1_2018.txt,"We are off to a very strong start in FY 2018, ...",[cloud],[Cloud computing],True
42,MSFT_q1_2018.txt,The Windows 10 Fall Creators Update empowers p...,[mixed reality],"[Augmented reality, Virtual reality]",True
46,MSFT_q1_2018.txt,"This month, we also introduced Windows Mixed R...",[mixed reality],"[Augmented reality, Virtual reality]",True
49,MSFT_q1_2018.txt,Ford's designers are using Mixed Reality to bl...,[mixed reality],"[Augmented reality, Virtual reality]",True
...,...,...,...,...,...
29739,ADP_q4_2018.txt,Celergo complements and augments our current m...,[platform],[Digital Product],True
29743,ADP_q4_2018.txt,"Last quarter, with the assistance of the Trans...",[transformation],[Practices],True
29754,ADP_q4_2018.txt,While we're also being well ahead of our origi...,[transformation],[Practices],True
29851,ADP_q4_2018.txt,So even though we had some losses towards the ...,[platform],[Digital Product],True


In [ ]:
df.to_csv('filtered_sentences.csv', index=False)